## styletest

用来进行风格测试的模板，基于提示词和风格词的组合生成图片，然后对比结果。

In [8]:
import json
from urllib import request, parse
import random
import uuid
import urllib.request
import urllib.parse
import io
import os
import time
from PIL.PngImagePlugin import PngInfo
import hashlib
from urllib.error import URLError


In [9]:
from enum import Enum, unique

@unique
class RunMode(Enum):
    LOCAL = 1
    POD_LOCAL = 2
    POD_REMOTE = 3

run_mode = RunMode.LOCAL
# 设置运行模式，来实现在本机，runpod本地，runpod远程的切换
server_address = "http://yc7sxv2vriceb8-3000.proxy.runpod.net" if run_mode == RunMode.POD_REMOTE else "0.0.0.0:3000" if run_mode == RunMode.POD_LOCAL else "127.0.0.1:8188"
client_id = str(uuid.uuid4())


In [10]:
def changeImageSize(width, height):
    # 最小的宽高为本机896，runpod1024
    limited = 896 if run_mode == RunMode.LOCAL else 1024
    minSize = min(width, height)

    # 缩放宽高，使得最窄的边等于limited
    if minSize > limited:
        scale = minSize / limited
        widthScale = int(width / scale)
        heightScale = int(height / scale)
    else:
        scale = limited / minSize
        widthScale = int(width * scale)
        heightScale = int(height * scale)

    # 将两个值都除以64，然后取整,也就是说边长必须是64的倍数
    widthScale = int(widthScale/64) * 64
    heightScale = int(heightScale/64) * 64

    return widthScale, heightScale

def checkImageExsit(imageDir, imageName):
    for file in os.listdir(imageDir):
        if file.startswith(imageName):
            return True
    return False


def queue_prompt(prompt):
    while True:
        try:
            p = {"prompt": prompt, "client_id": client_id}
            data = json.dumps(p).encode('utf-8')
            req = urllib.request.Request("http://{}/prompt".format(server_address), data=data)
            return json.loads(urllib.request.urlopen(req, timeout=5).read())
        except URLError:
            print("连接超时，正在重试...")

def get_history(prompt_id):
    with urllib.request.urlopen("http://{}/history/{}".format(server_address, prompt_id)) as response:
        return json.loads(response.read())

def get_image(filename, subfolder, folder_type):
    data = {"filename": filename, "subfolder": subfolder, "type": folder_type}
    url_values = urllib.parse.urlencode(data)
    with urllib.request.urlopen("http://{}/view?{}".format(server_address, url_values)) as response:
        return response.read()

def handle_whitespace(string: str):
    return string.strip().replace("\n", " ").replace("\r", " ").replace("\t", " ")

def parse_name(ckpt_name):
    path = ckpt_name
    filename = path.split("/")[-1]
    filename = filename.split(".")[:-1]
    filename = ".".join(filename)
    return filename


def calculate_sha256(file_path):
    sha256_hash = hashlib.sha256()

    with open(file_path, "rb") as f:
        # Read the file in chunks to avoid loading the entire file into memory
        for byte_block in iter(lambda: f.read(4096), b""):
            sha256_hash.update(byte_block)

    return sha256_hash.hexdigest()

def get_images(ws, prompt):
    prompt_id = queue_prompt(prompt)['prompt_id']
    output_images = {}
    while True:
        out = ws.recv()
        if isinstance(out, str):
            message = json.loads(out)
            if message['type'] == 'executing':
                data = message['data']
                if data['node'] is None and data['prompt_id'] == prompt_id:
                    break #Execution is done
        else:
            continue #previews are binary data

    history = get_history(prompt_id)[prompt_id]
    for o in history['outputs']:
        for node_id in history['outputs']:
            node_output = history['outputs'][node_id]
            if 'images' in node_output:
                images_output = []
                for image in node_output['images']:
                    image_data = get_image(image['filename'], image['subfolder'], image['type'])
                    images_output.append(image_data)
                output_images[node_id] = images_output

    return output_images
    
def saveImages(result, imageDir,imageName,comment,previewImage=0): 
    metadata = PngInfo()
    metadata.add_text("parameters", comment)  
    k = 0 
    for node_id in result:    
        if k < previewImage:
            k += 1
            continue    
        for image_data in result[node_id]:            
            image = Image.open(io.BytesIO(image_data))
            imagePath = imageDir + "/" + imageName + f"_{k}" + ".png"
            image.save(imagePath,pnginfo=metadata) 
            k += 1
    

## import json file

In [11]:
promptJson = 'api.json'

with open(promptJson) as f:
    prompt = json.load(f)

In [12]:
# 提示词文件名，通过变换这个文件名，可以实现不同的输出，文件名必须与在assets/texts下的文件名一致
fileName = "descriptions"

# prompts文件
promptsFile = f'../../assets/texts/{fileName}.txt'
with open(promptsFile, encoding='utf-8') as f:
    prompt_list = f.readlines()

stylesFile = '../../assets/texts/styles.txt'
with open(stylesFile, encoding='utf-8') as f:
    style_list = f.readlines()

style_list = [x.strip() for x in style_list]
# 删除所有为空的样式
style_list = [x for x in style_list if x != '']
# 删除所有不包含{input_text}的样式
style_list = [x for x in style_list if 'input_text' in x]

# 设置图片保存路径
imageDir = f"f:/Poledriver/Remake/{fileName}" if run_mode != RunMode.POD_LOCAL else f"../../../image-outputs/{fileName}"
# 如果文件夹不存在，创建文件夹
if not os.path.exists(imageDir):
    os.mkdir(imageDir)


In [13]:

from PIL import Image
from sdparsers import ParserManager
import websocket

ws = websocket.WebSocket()
ws.connect("ws://{}/ws?clientId={}".format(server_address, client_id))

batch_size = 2

# 遍历提示词组
for i in range(0, len(prompt_list)):
        input_text = prompt_list[i]
        # 取..之间的内容作为prompt
        input_text = input_text.split(".")[1]
        input_text = f"analog film grain,(TOK:1.5),skin pores,BREAK {input_text}, she have black long hair,big ass." 
               
        prompt['48']['inputs']['text'] = input_text       
        
        # 读取原图片名字，去掉后缀.png
        originalImagName = f"奶茶多多_{i}"
        # 检测目标文件夹里是否已经存在该图片，如果存在，跳过
        if checkImageExsit(imageDir, originalImagName):
            print(f"{originalImagName}已经存在，跳过")
            continue   
        
        # 下面是确定实际生成图片的大小，种子，以及保存图片信息
        # 保存图片的信息使用的是38号节点，也就是Save Image w/Metadata节点 
        prompt['36']['inputs']['batch_size'] = batch_size
        steps = 50
        cfg = 5
        sampler_name = prompt['51']['inputs']['sampler_name']
        scheduler = prompt['51']['inputs']['scheduler']
        positive = prompt['48']['inputs']['text']
        negative = prompt['49']['inputs']['text']
        
        modelname = prompt['4']['inputs']['ckpt_name']
        basemodelname = parse_name(modelname)        
        ckpt_path = "e:/sdwebui/webui/models/Stable-diffusion/" + modelname if run_mode != RunMode.POD_LOCAL else "../../../ComfyUI/models/checkpoints/" + modelname
        modelhash = calculate_sha256(ckpt_path)[:10]
        
        seed = random.randint(0, 976242998978323)
        prompt['73']['inputs']['seed']= seed
        b1 = 1.0
        b2 = 1.1
        s1 = 0.9
        s2 = 0.9
        
        prompt['58']['inputs']['b1'] = b1
        prompt['58']['inputs']['b2'] = b2
        prompt['58']['inputs']['s1'] = s1
        prompt['58']['inputs']['s2'] = s2
        prompt['59']['inputs']['b1'] = b1
        prompt['59']['inputs']['b2'] = b2
        prompt['59']['inputs']['s1'] = s1
        prompt['59']['inputs']['s2'] = s2
        
        comment = f"{handle_whitespace(positive)}\nNegative prompt: {handle_whitespace(negative)}\nSteps: {steps}, Sampler: {sampler_name}{f'_{scheduler}' if scheduler != 'normal' else ''}, CFG Scale: {cfg}, Seed: {seed}, Size: {896}x{1152}, Model hash: {modelhash}, Model: {basemodelname},FreeU:({b1},{b2},{s1},{s2}), Version: ComfyUI"        
    
        print(f"生成图片的信息是：{comment}")
        
        # 连接api，生成并保存图片
        start_time = time.time()
        
        images = get_images(ws, prompt)   
        saveImages(images, imageDir, originalImagName, comment=comment,previewImage=0)     
        end_time = time.time()
        print(f"耗时{end_time-start_time}秒")       
        
            

生成图片的信息是：analog film grain,(TOK:1.5),skin pores,BREAK  A glamorous model in a glittering evening gown against a city skyline, she have black long hair,big ass.
Negative prompt: tattoo,nude,(naked:1.5),nsfw,embedding:negativeXL_D,
Steps: 50, Sampler: dpmpp_2m_sde_karras, CFG Scale: 5, Seed: 103297457925153, Size: 896x1152, Model hash: d79e975e9b, Model: fudukiMix_v15,FreeU:(1.0,1.1,0.9,0.9), Version: ComfyUI
耗时89.10637640953064秒
生成图片的信息是：analog film grain,(TOK:1.5),skin pores,BREAK  A bohemian beauty in a flowing floral dress in a sunlit meadow, she have black long hair,big ass.
Negative prompt: tattoo,nude,(naked:1.5),nsfw,embedding:negativeXL_D,
Steps: 50, Sampler: dpmpp_2m_sde_karras, CFG Scale: 5, Seed: 576263345393177, Size: 896x1152, Model hash: d79e975e9b, Model: fudukiMix_v15,FreeU:(1.0,1.1,0.9,0.9), Version: ComfyUI
耗时89.34065175056458秒
生成图片的信息是：analog film grain,(TOK:1.5),skin pores,BREAK  A chic fashionista in a sleek black jumpsuit on a city street, she have black long hair,

In [14]:
import pygame
pygame.init()
my_sound = pygame.mixer.Sound('../../assets/sound/download-complete.wav')
my_sound.play()